# Generating templates and recordings with MEArec

This notebook shows how to generate templates and recordings from scratch using MEArec.

The last part of the notebook showcases some plotting routines available through the package.

In [ ]:
import MEArec as mr
import MEAutility as mu
import yaml
from pprint import pprint
import matplotlib.pylab as plt
%matplotlib notebook

## Load default configuration files

First, let's load the default configuration of MEArec

In [ ]:
default_info, mearec_home = mr.get_default_config()
pprint(default_info)

## Generating and saving templates

In [ ]:
# define cell_models folder
cell_folder = default_info['cell_models_folder']
template_params = mr.get_default_templates_params()
pprint(template_params)

Now let's change a few parameters and generate templates.

In [ ]:
template_params['n'] = 10
template_params['probe'] = 'Neuronexus-32'
# the templates are not saved, but the intracellular simulations are saved in 'templates_folder'
tempgen = mr.gen_templates(cell_models_folder=cell_folder, params=template_params, n_jobs=13)
# this will take a few minutes...

The `tempgen` variable is a `TemplateGenerator` object. It contains the `templates`, `locations`, `rotations`, and `celltypes` of the generated templates.

In [ ]:
print('Templates shape', tempgen.templates.shape)
print('Sample locations', tempgen.locations[:3])
print('Sample rotations', tempgen.rotations[:3])
print('Sample cell types', tempgen.celltypes[:3])

We can now save the `TemplateGenerator` object in h5 format.

In [ ]:
# save templates in h5 format
mr.save_template_generator(tempgen, filename='data/test_templates.h5')

## Generating and saving recordings

Once the templates have been generated, we can use them to generate recordings. Let's fisrt load and take a look at the default parameters:

In [ ]:
recordings_params = mr.get_default_recordings_params()
pprint(recordings_params)

Similarly to the templates generation, we can change th eparameters that we pass to the `gen_recordings` function.
In this case we will keep the default parameters.

In [ ]:
recgen = mr.gen_recordings(templates='data/test_templates.h5', params=recordings_params)

The `recgen` variable is a `RecordingGenerator` object. It contains the selected `templates`, `template_locations`, `template_rotations`, and `template_celltypes`, as well as `spiketrains` and `recordings`.

In [ ]:
print('Recordings shape', recgen.recordings.shape)
print('Selected templates shape', recgen.recordings.shape)
print('Sample template locations', recgen.template_locations[:3])
print('Number of neurons', len(recgen.spiketrains))
print('Sample spike train', recgen.spiketrains[0].times)

We can now save the `RecordingGenerator` object to h5 format.

In [ ]:
# save recordings in h5 format
mr.save_recording_generator(recgen, filename='data/test_recordings.h5')

## Plotting the templates and recordings

`MEArec` implements some basic plotting functions.

In [ ]:
# plot templates
mr.plot_templates(recgen, single_axes=False, ncols=4)
mr.plot_templates(recgen, single_axes=True, cmap='rainbow')

In [ ]:
mr.plot_recordings(recgen)
mr.plot_recordings(recgen, start_time=0, end_time=1, overlay_templates=True)

In [ ]:
mr.plot_waveforms(recgen, electrode='max', cmap='rainbow')

In [ ]:
mr.plot_rasters(recgen.spiketrains)

In [ ]:
_ = mr.plot_pca_map(recgen, cmap='coolwarm')